# Segmenting and Clustering Neighborhoods in Toronto

**PART 1**

Create a dataframe: Even though the LAB does NOT explain ANYTHING about how to get the data in Wikipedia, I think I have to go to the following page:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

In [1]:
#I do not know what is that or if it is necessary:

!pip install bs4
!pip install lxml
!pip install html5lib


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=786aa4f240787c07202a65ae7fbd5131adf97b5012f2e54fb7e9fba21acd62b0
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.

In [2]:
from bs4 import BeautifulSoup
import requests
import lxml

import numpy as np
import pandas as pd


In [3]:
my_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Create a Beatiful soup object.

Create a table


In [4]:
soup =  BeautifulSoup(my_url,'html5lib')
table = soup.find('table_class',{'class':'wikitable sortable jquery-tablesorter '}) 

 Create a list: table_conttens

 Create a dictionary: cell. That dictionary will have 3 keys: PostalCode, Borough and Neighborhood.

 As postal code contains upto 3 characters extract that using tablerow.p.text

 Next use split ,strip and replace functions for getting Borough and Neighborhood information.

 Append to the list  

 Create a dataframe with list


In [5]:
#The following code appears in a link given in the LAB.
#It doesn't explain how you have to use it

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)



In [6]:
#Checking that we have obtained some data
table_contents

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'PostalCode': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'},
 {'PostalCode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront'},
 {'PostalCode': 'M6A',
  'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor, Lawrence Heights'},
 {'PostalCode': 'M7A',
  'Borough': "Queen's Park",
  'Neighborhood': 'Ontario Provincial Government'},
 {'PostalCode': 'M9A',
  'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue'},
 {'PostalCode': 'M1B',
  'Borough': 'Scarborough',
  'Neighborhood': 'Malvern, Rouge'},
 {'PostalCode': 'M3B',
  'Borough': 'North York',
  'Neighborhood': 'Don Mills North'},
 {'PostalCode': 'M4B',
  'Borough': 'East York',
  'Neighborhood': 'Parkview Hill, Woodbine Gardens'},
 {'PostalCode': 'M5B',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson'},
 {'PostalCode': 'M6B', 'Borough': 'North York', 'N

I convert the table_contens into a pandas dataframe:

In [6]:

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.shape


(103, 3)

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned:

In [7]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace=True)
df = df.reset_index(drop=True)
df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [8]:

#Using groupby

df_postcode = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index() 
df_postcode.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
#Cheking the info from my dataframe
df_postcode.info()

#I want to be sure that Borough is not always Scarborough so I see how many different Boroughs the Dataframe has
df_postcode.groupby('Borough').count()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PostalCode    103 non-null    object
 1   Borough       103 non-null    object
 2   Neighborhood  103 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


,PostalCode,Neighborhood
Borough,,
Central Toronto,9,9
Downtown Toronto,17,17
Downtown Toronto Stn A,1,1
East Toronto,4,4
East Toronto Business,1,1
East York,4,4
East York/East Toronto,1,1
Etobicoke,11,11
Etobicoke Northwest,1,1


•	If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [10]:
#Count NaN values int the column Neighborhood

df_postcode['Neighborhood'].isna().sum()

#Conclussion: there are 0 boroughs with empty neighborhood

0

•	In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [12]:
df_postcode.shape

(103, 3)

**End of part 1**
***

**PART 2**

We need to get the latitude and the longitude coordinates of each neighborhood. 

The following cell will take a long while to execute

In [11]:
!conda install -c conda-forge geocoder -y


import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

#I do not know if that is going to work but after getting this error:
#NameError: name 'postal_code' is not defined
#I have tried the following, before the loop
#postal_code=[]
#However, when I do that the loop never finish the execution

#After that I substitued postal_code in the loop for M5G: same error



Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0      

pandoc-2.12          | 12.4 MB   | ##################################### | 100% 
libnghttp2-1.43.0    | 808 KB    | ##################################### | 100% 
typing_extensions-3. | 25 KB     | ##################################### | 100% 
pymssql-2.1.5        | 264 KB    | ##################################### | 100% 
pygments-2.8.1       | 736 KB    | ##################################### | 100% 
threadpoolctl-2.1.0  | 15 KB     | ##################################### | 100% 
retrying-1.3.3       | 11 KB     | ##################################### | 100% 
pytz-2021.1          | 239 KB    | ##################################### | 100% 
astropy-4.2.1        | 7.5 MB    | ##################################### | 100% 
nbformat-5.1.3       | 47 KB     | ##################################### | 100% 
pytables-3.6.1       | 1.5 MB    | ##################################### | 100% 
jmespath-0.10.0      | 21 KB     | ##################################### | 100% 
pooch-1.3.0          | 40 KB

readline-8.1         | 295 KB    | ##################################### | 100% 
libclang-11.1.0      | 19.2 MB   | ##################################### | 100% 
pandocfilters-1.4.2  | 9 KB      | ##################################### | 100% 
botocore-1.20.55     | 4.6 MB    | ##################################### | 100% 
click-7.1.2          | 64 KB     | ##################################### | 100% 
ibm-wsrt-py37main-ma | 2 KB      | ##################################### | 100% 
lzo-2.10             | 314 KB    | ##################################### | 100% 
_openmp_mutex-4.5    | 5 KB      | ##################################### | 100% 
ptyprocess-0.7.0     | 16 KB     | ##################################### | 100% 
cx_oracle-8.1.0      | 193 KB    | ##################################### | 100% 
bokeh-2.3.1          | 8.2 MB    | ##################################### | 100% 
gast-0.4.0           | 12 KB     | ##################################### | 100% 
pthread-stubs-0.4    | 5 KB 

done


In [14]:
type(lat_lng_coords)

NoneType

In [15]:
#df_postcode ['PostalCode'] - Así obtengo la columna de PostalCode
#The next code will produce None as a result.

#g = geocoder.google('M1B, Toronto, Ontario')
#print (g.latlng)

#Therefore I substitute google for arcgis, so I obtain some result
g = geocoder.arcgis('M1B, Toronto, Ontario')
print (g.latlng)

[43.811390000000074, -79.19661999999994]


That is an example I found in internet of how they use geocoder

import geocoder

g = geocoder.google('Mountain View, CA')

g.latlng

(37.3860517, -122.0838511)


**The following are the instructios for a loop until you get the coordinates**


while(lat_lng_coords is None):

  >g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
  
  >lat_lng_coords = g.latlng


latitude = lat_lng_coords[0]

longitude = lat_lng_coords[1]

In [16]:
#iloc[row][column]--column 0(PostalCode) must be fix
#df_postcode.iloc[10][0]


#I try to get two variables: latitude and longitude. It works
g = geocoder.arcgis('{}, Toronto, Ontario'.format(df_postcode.iloc[11][0]))
lat_lng_coords = g.latlng
lat = [lat_lng_coords[0]]
long = [lat_lng_coords[1]]

print (type(lat))
print (lat, long)


#I try to add more values to latitude  and longitude. It works
g = geocoder.arcgis('{}, Toronto, Ontario'.format(df_postcode.iloc[12][0]))
lat_lng_coords = g.latlng
lat.append(lat_lng_coords[0])
long.append (lat_lng_coords[1])

print (lat, long)

<class 'list'>
[43.75075000000004] [-79.30053999999996]
[43.75075000000004, 43.794520000000034] [-79.30053999999996, -79.26707999999996]


In the next cells I will obtain two list: latitude list and longitude list:

In [12]:
#Next step is figure out how I can use a loop to fill latitude and longitude with values
#The loop has to go from value 0 to 102 in the PostalCode column from my dataframe (df_postcode)
latitude = []
longitude = []

for x in range(0, 103):
   g = geocoder.arcgis('{}, Toronto, Ontario'.format(df_postcode.iloc[x][0]))
   lat_lng_coords = g.latlng
   latitude.append(lat_lng_coords[0])
   longitude.append (lat_lng_coords[1])

In [13]:
#I want to check if lat and long have elements

print (len(latitude))
print (len(longitude))

#I chechk the first elements
latitude[:10]

103
103


[43.811390000000074,
 43.78574000000003,
 43.765750000000025,
 43.76812000000007,
 43.76944000000003,
 43.74446000000006,
 43.725820000000056,
 43.71289000000007,
 43.72360000000003,
 43.695100000000025]

I will complete my data frame (df_postcode) with two new columns: Latitude and Longitude

In [14]:
#Following I need to add two columns to the dataframe
df_postcode['Latitude'] = latitude
df_postcode['Longitude'] = longitude

In [15]:
df_postcode.shape
df_postcode.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


**End of part 2**
***

**PART 3**

In [16]:
import requests # library to handle requests

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import json # library to handle JSON files
! pip install geopy

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37ha70c682_1
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy   

In [17]:
!pip install -U numpy
!pip install -U pandas
!pip install -U scipy
!pip install -U scikit-learn
!pip install -U imbalanced-learn

     |████████████████████████████████| 27.4 MB 14.6 MB/s eta 0:00:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.3
    Uninstalling scipy-1.5.3:
      Successfully uninstalled scipy-1.5.3
     |████████████████████████████████| 206 kB 15.5 MB/s eta 0:00:01


Let's see how many Boroghs has Toronto

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_postcode['Borough'].unique()),
        df_postcode.shape[0]
    )
)


The dataframe has 15 boroughs and 103 neighborhoods.


In [19]:
#Creating a function
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)


1. We want to obtain the number of venues per zone. We need to use nearby_venues.shape[0]

2. Therefore we need first, to know how we can obtain the venues.

venues =results['response']['groups'][0]['items']

3. For obtaining the venues we had to get results. Those results come from a request we make to foursquare.

These results are obtained with an url we have to create, and they come in a form of json.

results = requests.get(url).json()

We also have to work on the json file (therefore I imported json_normalize from pandas:

nearby_venues = json_normalize(venues) # flatten JSON

4. We have to create the url. For the url we need that data

CLIENT_ID: VH5TAWD0V5XFZ5NTFM1OWR54NH2Z2LJG2C3NEHJHD0UHSGLB

CLIENT_SECRET:0DUAUPY1JBCRWUQFOYF3Z02N0BLJCYIHE4MM504SOXDZ2D03

5.For the url we need to give latitude and longitude

neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value

neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value



In [20]:
#Creating variables for the url
CLIENT_ID = 'VH5TAWD0V5XFZ5NTFM1OWR54NH2Z2LJG2C3NEHJHD0UHSGLB' # your Foursquare ID
CLIENT_SECRET = '0DUAUPY1JBCRWUQFOYF3Z02N0BLJCYIHE4MM504SOXDZ2D03' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: VH5TAWD0V5XFZ5NTFM1OWR54NH2Z2LJG2C3NEHJHD0UHSGLB
CLIENT_SECRET:0DUAUPY1JBCRWUQFOYF3Z02N0BLJCYIHE4MM504SOXDZ2D03


In [21]:
#Create  neighborhood_latitude and longitude for the url
neighborhood_latitude = df_postcode.loc[0, 'Latitude']
neighborhood_longitude = df_postcode.loc[0, 'Longitude']

In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=VH5TAWD0V5XFZ5NTFM1OWR54NH2Z2LJG2C3NEHJHD0UHSGLB&client_secret=0DUAUPY1JBCRWUQFOYF3Z02N0BLJCYIHE4MM504SOXDZ2D03&v=20180605&ll=43.811390000000074,-79.19661999999994&radius=500&limit=100'

In [23]:
#We will store the venues json file in results
results = requests.get(url).json()
results

#results = requests.get(url).json()['response']['groups'][0]['items'][0]

{'meta': {'code': 200, 'requestId': '60813d18e56eec0c3f7592e5'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.815890004500076,
    'lng': -79.19039569057789},
   'sw': {'lat': 43.80688999550007, 'lng': -79.20284430942199}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label':

In [45]:
#That was an attempt to get a normal result, instead of getting a list of 1 venue:
#venues = results['response']['venues']
#dataframe = json_normalize(venues) 
#dataframe.head()

#It didn't work

In [25]:
results.items()
results.keys()

dict_keys(['meta', 'response'])

In [29]:
venues = results['response']['groups'][0]['items']
#venues = results['response']['venues']

In [30]:
print('That is the info for results:')
print (type(results))
print (len(results))
print ('That is the info for venues')
print (type(venues))
print (len(venues))

That is the info for results:
<class 'dict'>
2
That is the info for venues
<class 'list'>
1


 def get_nearby_venues(names,latitudes,longitudes,radius=500):
    
    venue_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&oauth_token={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            ACCESS_TOKEN,
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items'][0]
   
        
        venue_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng']) for v in results])
            
        nearby_venues = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude']
                  
        
        
        return(nearby_venues)

get_nearby_venues(names=data['Neighborhood'] , latitudes=data["Latitude"], longitudes=data['Longitude'])

In [31]:
pd.DataFrame(venues)

,reasons,venue,referralId
0,"{'count': 0, 'items': [{'summary': 'This spot ...","{'id': '4bb6b9446edc76b0d771311c', 'name': 'We...",e-0-4bb6b9446edc76b0d771311c-0


In [55]:
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

In [56]:
#I've just discovered something went wrong with my dataframe. It only has one row

print(type(nearby_venues))
nearby_venues.shape

<class 'pandas.core.frame.DataFrame'>


(1, 4)

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [32]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

AttributeError: 'Series' object has no attribute '_mgr'

In [ ]:
#print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))